# <p style="color:dodgerblue">01 Create Rekognition Backend</p>
This stack creates backend services and resources that are used by the website/app to invoke rekognition face liveness
  
(Jupyter Notebook developed with Kernel Python 3.11.6)
<hr style="border:1px dotted; color:floralwhite">

# <span style="color:deeppink">GETTING STARTED</span>
# Local client requirements for this Lab (macOS)
- *See <span style="color:gold">Appendix - Jupyter Install Requirements (macOS)</span> at the bottom of this lab to install macOS requirements, windows requirements will be similar, apart from Homebrew.*  
- These requirements are generic and allow you to run Python notebooks, use Boto3, etc - they are simply to get your local environment in a state that can support Jupyter Notebooks and not specific to Rekognition

<hr style="border:1px dotted">
<hr style="border:1px dotted;color:greenyellow">

# <p style="color:DarkTurquoise">Rekognition Face Liveness Prerequisites</p>
### <p style="color:DarkTurquoise">NOTE we are using us-east-1</p>
To develop applications that use Rekognition and Rekognition Face Liveness, we require the following resources to be provisioned:
  
- AWS Amplify CLI - needed to be able to develop the Rekognition Face Liveness services
  - https://docs.aws.amazon.com/rekognition/latest/dg/face-liveness-prerequisites.html#face-liveness-prerequisites-amplify
  - After you have installed CLI, complete the Configure Auth step steps seen at the Amplify UI docs site to set up your AWS Amplify resources.
  - https://ui.docs.amplify.aws/react/connected-components/liveness#step-1-configure-auth



<hr style="border:1px dotted;color:DarkTurquoise">
<hr style="border:1px dotted;color:greenyellow">

# <p style="color:greenyellow">Now we are going to create the backend services</p>

# <p style="color:greenyellow">Lets Create Clients and Variables</p>
- We do these setup cells here because we can then use the vars and clients to clean up resources later without having to run multiple cells if we lose the kernel  
  
-  <span style="color:greenyellow">Please note we use us-east-1 region as Rekognition Face Liveness regions are limited<span>

In [ ]:
import boto3
import json
import random

# region - we use us-east-1 as Bedrock is limited in other reasons
myRegion='us-east-1'
myAccountNumber = boto3.client("sts").get_caller_identity()["Account"]

# set up a boto3 session using a profile that is able to create services in the region
# this is typically a developer profile or deployment profile
# we DO NOT need the amplify profile we crerated above yet - that is used in stack 02
sessionBoto3 = boto3.Session(profile_name="default", region_name=myRegion)

# names for services we will create below
# s3 bucket - MUST BE A UNIQUE NAME
myBucketRekognition='doit-rekognition-bucket-' + str(random.randint(0, 1000)) + '-' + str(random.randint(0, 1000))

# iam
myRoleLambda1="doit-rekognition-lambda-start-liveness-session-role"
myPolicyLambda1="doit-rekognition-lambda-start-liveness-session-policy"
myRoleLambda1ARN='RETRIEVED FROM ROLE BELOW ONCE CREATED'
myRoleLambda2="doit-rekognition-lambda-liveness-session-result-role"
myPolicyLambda2="doit-rekognition-lambda-liveness-session-result-policy"
myRoleLambda2ARN='RETRIEVED FROM ROLE BELOW ONCE CREATED'

# lambda
myLambda1='doit-rekognition-start-liveness-session-fn'
myLambda2='doit-rekognition-liveness-session-result-fn'
myLambda1ARN='RETRIEVED FROM LAMBDA BELOW ONCE QUERIED'
myLambda2ARN='RETRIEVED FROM LAMBDA BELOW ONCE QUERIED'

# api gateway
myAPIGateway='doit-rekognition-api'
myResource1='start-liveness-session'
myResource2='liveness-session-result'
myAPIid='RETRIEVED FROM LAMBDA BELOW ONCE QUERIED'

print ('Done! Move to the next cell ->')

In [ ]:
# local client path for resources
myLocalPathForResources='/Users/simondavies/Documents/GitHub/labs/rekognition/face liveness/resources/'

print ('Done! Move to the next cell ->')

- create required clients

In [ ]:
# s3
s3 = sessionBoto3.client('s3')

# rekognition
rekognition = sessionBoto3.client('rekognition')

# iam
iam = sessionBoto3.client('iam')

# lambda
lambdac = sessionBoto3.client('lambda')

# logs (cloudwatch)
logs = sessionBoto3.client('logs')

# api gateway
apiGateway = sessionBoto3.client('apigateway')

print ('Done! Move to the next cell ->')

- tags for all services that are created - you can never have too many tags!
  - make sure you have a tagging policy in place

In [ ]:
# define tags added to all services we create
myTags = [
    {"Key": "env", "Value": "non_prod"},
    {"Key": "owner", "Value": "doit_lab"},
    {"Key": "project", "Value": "doit_rekognition-face-liveness-lab"},
    {"Key": "author", "Value": "simon"},
]
myTagsDct = {
    "env": "non_prod",
    "owner": "doit_lab",
    "project": "doit_rekognition-face-liveness-lab",
    "author": "simon",
}

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:greenyellow">
<hr style="border:1px dotted;color:crimson">

# <p style="color:crimson">Create S3 Bucket</p>
- defaults used, will use sse-s3 encryption and block public access

In [ ]:
# create bucket
if (myRegion != 'us-east-1'):
    s3.create_bucket(
        Bucket=myBucketRekognition, CreateBucketConfiguration={"LocationConstraint": myRegion}
    )
else:
    s3.create_bucket(
        Bucket=myBucketRekognition
    )

s3.put_bucket_tagging(Bucket=myBucketRekognition, Tagging={"TagSet": myTags})

# create a "folder" - really keys as S3 is flat
s3.put_object(Bucket=myBucketRekognition, Key="images/")

# Define the policy for rekognition face liveness to store images
bucket_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowRekognitionToWriteToS3",
            "Effect": "Allow",
            "Principal": {
                "Service": "rekognition.amazonaws.com"
            },
            "Action": "s3:PutObject",
            "Resource": f"arn:aws:s3:::{myBucketRekognition}/*"
        }
    ]
}
bucket_policy = json.dumps(bucket_policy)
s3.put_bucket_policy(Bucket=myBucketRekognition, Policy=bucket_policy)

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:crimson">
<hr style="border:1px dotted;color:orchid">

# <p style="color:orchid">Create IAM</p>
- roles and policies for the services to interact with other services

In [ ]:
# myRoleLambda1
# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

# define inline policy
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
            ],
            "Resource": "*",
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:GetBucketLocation"
            ],
            "Resource": [
                f"arn:aws:s3:::{myBucketRekognition}",
                f"arn:aws:s3:::{myBucketRekognition}/*"
            ]
        }
    ],
}

# create inline policy
policy = iam.create_policy(
    PolicyName=myPolicyLambda1,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy for {} lambda".format(myLambda1),
    Tags=[
        *myTags,
    ],
)

# create role
role = iam.create_role(
    RoleName=myRoleLambda1,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Role for {} lambda".format(myLambda1),
    Tags=[
        *myTags,
    ],
)

# attach managed policies to role
iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], 
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaVPCAccessExecutionRole'
)
iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], 
    PolicyArn='arn:aws:iam::aws:policy/AmazonRekognitionFullAccess'
)

# attach inline policies to role
response = iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], PolicyArn=policy["Policy"]["Arn"]
)

myRoleLambda1ARN = role['Role']['Arn']

print ('Done! Move to the next cell ->')

In [ ]:
# myRoleLambda2
# trust policy for the role
roleTrust = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

# define inline policy
policyJson = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents",
            ],
            "Resource": "*",
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:GetBucketLocation"
            ],
            "Resource": [
                f"arn:aws:s3:::{myBucketRekognition}",
                f"arn:aws:s3:::{myBucketRekognition}/*"
            ]
        }
    ],
}

# create inline policy
policy = iam.create_policy(
    PolicyName=myPolicyLambda2,
    PolicyDocument=json.dumps(policyJson),
    Description="Policy for {} lambda".format(myLambda2),
    Tags=[
        *myTags,
    ],
)

# create role
role = iam.create_role(
    RoleName=myRoleLambda2,
    AssumeRolePolicyDocument=json.dumps(roleTrust),
    Description="Role for {} lambda".format(myLambda2),
    Tags=[
        *myTags,
    ],
)

# attach managed policies to role
iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], 
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaVPCAccessExecutionRole'
)
iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], 
    PolicyArn='arn:aws:iam::aws:policy/AmazonRekognitionFullAccess'
)

# attach inline policies to role
response = iam.attach_role_policy(
    RoleName=role["Role"]["RoleName"], PolicyArn=policy["Policy"]["Arn"]
)

myRoleLambda2ARN = role['Role']['Arn']

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:orchid">
<hr style="border:1px dotted;color:LightSkyBlue">

# <span style="color:LightSkyBlue">Create Lambda</span>
- You need to create the zip file from the lambda resource folder as create lambda function requires a zipped file
- You can do this from a terminal window as long as you have cd'ed to the folder that contains the function code
  - Eg from the folder that contains the lambda function code (and all of the libraries if any are required) ...
    - NOTE the below only zips up a single file, if your lambda requires libraries put the lambda in its own folder with the libraries in their folders, and specify .* instead of the .py lambda file name
    - *zip -r doit-rekognition-start-liveness-session-fn.zip doit-rekognition-start-liveness-session-fn.py*
    - *zip -r doit-rekognition-liveness-session-result-fn.zip doit-rekognition-liveness-session-result-fn.py*

In [ ]:
# define vars
myLambdaZip='{}lambda/{}.zip'.format(myLocalPathForResources, myLambda1)

# Loads the zip file as binary code. 
with open(myLambdaZip, 'rb') as f: 
    code = f.read()
    
# create lambda
myLambdaFunction = lambdac.create_function(
    FunctionName=myLambda1,
    Runtime='python3.12',
    Role=myRoleLambda1ARN,
    Handler='{}.lambda_handler'.format(myLambda1),
    Code={'ZipFile':code},
    Description='Starts a rekognition face liveness session',
    Timeout=30,
    MemorySize=128,
    Publish=True,
    PackageType='Zip',
    Environment={
        'Variables': {
            'S3': myBucketRekognition
        }
    },
    Tags=myTagsDct,
    Architectures=[
        'x86_64',
    ],
    LoggingConfig={
        'LogFormat': 'JSON',
        'ApplicationLogLevel': 'INFO',
        'SystemLogLevel': 'WARN'
    }
)

myLambda1ARN=myLambdaFunction['FunctionArn']

print ('Done! Move to the next cell ->')

In [ ]:
logs.create_log_group(
    logGroupName='/aws/lambda/' + myLambda1,
    tags=myTagsDct
)

print ('Done! Move to the next cell ->')

In [ ]:
# define vars
myLambdaZip='{}lambda/{}.zip'.format(myLocalPathForResources, myLambda2)

# Loads the zip file as binary code. 
with open(myLambdaZip, 'rb') as f: 
    code = f.read()
    
# create lambda
myLambdaFunction = lambdac.create_function(
    FunctionName=myLambda2,
    Runtime='python3.12',
    Role=myRoleLambda2ARN,
    Handler='{}.lambda_handler'.format(myLambda2),
    Code={'ZipFile':code},
    Description='Gets the result of a rekognition face liveness session',
    Timeout=30,
    MemorySize=128,
    Publish=True,
    PackageType='Zip',
    Tags=myTagsDct,
    Architectures=[
        'x86_64',
    ],
    LoggingConfig={
        'LogFormat': 'JSON',
        'ApplicationLogLevel': 'INFO',
        'SystemLogLevel': 'WARN'
    }
)

myLambda2ARN=myLambdaFunction['FunctionArn']

print ('Done! Move to the next cell ->')

In [ ]:
logs.create_log_group(
    logGroupName='/aws/lambda/' + myLambda2,
    tags=myTagsDct
)

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:orchid">
<hr style="border:1px dotted;color:DarkSeaGreen">

# <p style="color:DarkSeaGreen">Create API</p>
Creates a rest API and methods to perform face liveness detection called by the app/website

In [ ]:
# Create the REST API
api = apiGateway.create_rest_api(
    name=myAPIGateway,
    description="API for Rekognition Liveness Detection",
    endpointConfiguration={"types": ["REGIONAL"]},
)
myAPIid = api["id"]

# Get the root resource ID
resources = apiGateway.get_resources(restApiId=myAPIid)
rootId = next(item for item in resources["items"] if item["path"] == "/")["id"]

print ('Done! Move to the next cell ->')

In [ ]:
# Create the resource for myLambda1ARN
resource = apiGateway.create_resource(
    restApiId=myAPIid, parentId=rootId, pathPart=myResource1
)
resourceId = resource["id"]

# Create the method 
httpMethod = "GET"
integrationHttpMethod = "POST"

apiGateway.put_method(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    authorizationType="NONE",  # Change this if you need authorization
)

# Set up the integration
apiGateway.put_integration(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    type="AWS_PROXY",
    integrationHttpMethod=integrationHttpMethod,
    uri=f"arn:aws:apigateway:{apiGateway.meta.region_name}:lambda:path/2015-03-31/functions/{myLambda1ARN}/invocations",
)

# Set up method response
apiGateway.put_method_response(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    statusCode="200",
    responseParameters={
        "method.response.header.Access-Control-Allow-Origin": True,
    },
)
apiGateway.put_method_response(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    statusCode="500",
    responseParameters={
        "method.response.header.Access-Control-Allow-Origin": True,
    },
    responseModels={"application/json": "Error"},
)

# Grant API Gateway permission to invoke the Lambda function
lambdac.add_permission(
    FunctionName=myLambda1,
    StatementId=f"apigateway-invoke-{httpMethod}",
    Action="lambda:InvokeFunction",
    Principal="apigateway.amazonaws.com",
    SourceArn=f"arn:aws:execute-api:{apiGateway.meta.region_name}:{myAccountNumber}:{myAPIid}/*/{httpMethod}/{myResource1}",
)

print ('Done! Move to the next cell ->')

In [ ]:
# Create the resource for myLambda2ARN
resource = apiGateway.create_resource(
    restApiId=myAPIid, parentId=rootId, pathPart=myResource2
)
resourceId = resource["id"]

# Create the method 
httpMethod = "GET"
integrationHttpMethod = "POST"

apiGateway.put_method(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    authorizationType="NONE",  # Change this if you need authorization
)

# Set up the integration
apiGateway.put_integration(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    type="AWS_PROXY",
    integrationHttpMethod=integrationHttpMethod,
    uri=f"arn:aws:apigateway:{apiGateway.meta.region_name}:lambda:path/2015-03-31/functions/{myLambda2ARN}/invocations",
)

# Set up method response
apiGateway.put_method_response(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    statusCode="200",
    responseParameters={
        "method.response.header.Access-Control-Allow-Origin": True,
    },
)
apiGateway.put_method_response(
    restApiId=myAPIid,
    resourceId=resourceId,
    httpMethod=httpMethod,
    statusCode="500",
    responseParameters={
        "method.response.header.Access-Control-Allow-Origin": True,
    },
    responseModels={"application/json": "Error"},
)

# Grant API Gateway permission to invoke the Lambda function
lambdac.add_permission(
    FunctionName=myLambda2,
    StatementId=f"apigateway-invoke-{httpMethod}",
    Action="lambda:InvokeFunction",
    Principal="apigateway.amazonaws.com",
    SourceArn=f"arn:aws:execute-api:{apiGateway.meta.region_name}:{myAccountNumber}:{myAPIid}/*/{httpMethod}/{myResource2}",
)

print ('Done! Move to the next cell ->')

In [ ]:
# Create a deployment for the api
stageName = 'poc'
deployment = apiGateway.create_deployment(restApiId=myAPIid, stageName=stageName)

print(f'Update apiGatewayUrl var in app.js with the following:\nhttps://{myAPIid}.execute-api.{myRegion}.amazonaws.com/{stageName}')

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:DarkSeaGreen">
<hr style="border:1px dotted;color:deeppink">

# <p style="color:deeppink">STACK 01 COMPLETE!</p>

<hr style="border:1px dotted;color:deeppink">
<hr style="border:1px dotted;color:MediumSpringGreen">

# <p style="color:MediumSpringGreen">Go to Stack 02</p>
Stack 02 will create the react js website with face liveness stack, including the Amplify requirements and Cognito auth.

<hr style="border:1px dotted;color:MediumSpringGreen">
<hr style="border:1px dotted;color:orangered">
<hr style="border:1px dotted;color:orangered">
<hr style="border:1px dotted;color:orangered">

# <p style="color:orangered">CLEAN UP!!</p>
# <p style="color:orangered">DO NOT RUN THESE UNLESS YOU WANT TO DESTROY EVERYTHING</p>
- If you have lost the Kernel:
  - Run the cells contained in the <span style="color:greenyellow">Set Up Requirements<span> section before continuing...
  - Any IDs or ARNs will have to be manually stated

In [ ]:
# api gateway permissions
try:
    lambdac.remove_permission(FunctionName=myLambda1, StatementId=f"apigateway-invoke-{httpMethod}")
    lambdac.remove_permission(FunctionName=myLambda2, StatementId=f"apigateway-invoke-{httpMethod}")
except Exception as err:
    print(f'1:{err}')

print ('Done! Move to the next cell ->')

In [ ]:
# api gateway
apiGateway.delete_rest_api(restApiId=myAPIid)

print ('Done! Move to the next cell ->')

In [ ]:
# lambdas
lambdac.delete_function(FunctionName=myLambda1)
lambdac.delete_function(FunctionName=myLambda2)

print ('Done! Move to the next cell ->')

In [ ]:
# delete roles and policies
try:
    iam.detach_role_policy(
        RoleName=myRoleLambda1, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda1)
    )
except Exception as err:
    print(f'1:{err}')

try:
    iam.detach_role_policy(
        RoleName=myRoleLambda1, 
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaVPCAccessExecutionRole'
    )
except Exception as err:
    print(f'2:{err}')

try:
    iam.detach_role_policy(
        RoleName=myRoleLambda1, 
        PolicyArn='arn:aws:iam::aws:policy/AmazonRekognitionFullAccess'
    )
except Exception as err:
    print(f'3:{err}')

try:
    iam.delete_role(RoleName=myRoleLambda1)
except Exception as err:
    print(f'4:{err}')

try:
    iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda1))
except Exception as err:
    print(f'5:{err}')

try:
    iam.detach_role_policy(
        RoleName=myRoleLambda2, PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda2)
    )
except Exception as err:
    print(f'6:{err}')

try:
    iam.detach_role_policy(
        RoleName=myRoleLambda2, 
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaVPCAccessExecutionRole'
    )
except Exception as err:
    print(f'7:{err}')

try:
    iam.detach_role_policy(
        RoleName=myRoleLambda2, 
        PolicyArn='arn:aws:iam::aws:policy/AmazonRekognitionFullAccess'
    )
except Exception as err:
    print(f'8:{err}')

try:
    iam.delete_role(RoleName=myRoleLambda2)
except Exception as err:
    print(f'9:{err}')

try:
    iam.delete_policy(PolicyArn='arn:aws:iam::{}:policy/{}'.format(myAccountNumber, myPolicyLambda2))
except Exception as err:
    print(f'10:{err}')

print ('Done! Move to the next cell ->')

In [ ]:
# log group and log streams
logs.delete_log_group(
    logGroupName='/aws/lambda/' + myLambda1
)

logs.delete_log_group(
    logGroupName='/aws/lambda/' + myLambda2
)

print ('Done! Move to the next cell ->')

In [ ]:
# delete s3 bucket
# NOTE WARNING - this will delete all objects in the bucket with NO prompt or confirmation
try:
    s3r = boto3.resource('s3')
    bucket = s3r.Bucket(myBucketRekognition)
    bucket.objects.all().delete()
except Exception as err:
    print(f'9:{err}')

try:
    # delete the bucket
    response = s3.delete_bucket(Bucket=myBucketRekognition)
except Exception as err:
    print(f'9:{err}')

print ('Done! Move to the next cell ->')

<hr style="border:1px dotted;color:coral">
<hr style="border:1px dotted;color:coral">
<hr style="border:1px dotted;color:coral">
<hr style="border:1px dotted;color:gold">
<hr style="border:1px dotted;color:gold">
<hr style="border:1px dotted;color:gold">

# <p style="color:gold">Appendix - Jupyter Install Requirements (macOS)</p>
#### <p style="color:deeppink">- If you are running VSCode on a laptop, follow all of below.<br>- If you are running Jupyter inside an AWS Account, you don't need to do anything!</p>

  - Credentials to the AWS account this notebook executes in is provided by AWS configure
  - You must already have an IAM user with code (Command Line Interface) access and AWS access keys to be able to use these credentials in AWS configure  
    
  - arn:aws:iam::###########:user/simon-davies-cli created for this lab  

### <p style="color:gold">1. Homebrew</p> 
If you haven't installed Homebrew, you can install it by running the following command here or in the terminal:

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1.1 Virtual Environments</p> 
- You can create a virtual environment that ensures any libraries you install are restricted to the venv.
  - https://code.visualstudio.com/docs/python/environments
- To enable the virtual environment once you have created it, ensure you open the folder in vs code rather than individual files.

In [ ]:
%%bash
sudo /bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"

### <p style="color:gold">1.2 Python</p> 
Once Homebrew is installed, you can install Python using the following command  
*check what you have before installing/upgrading*  
*you will need to quit and restart vsCode to use python once installed (or updated)*

In [ ]:
%%bash
python3 --version
which python3

In [ ]:
%%bash
brew install python

### <p style="color:gold">2. boto3 and other Python requirements</p> 
* boto3 must be installed on your client
  * *Boto3 is the Amazon Web Services (AWS) Software Development Kit (SDK) for Python, which allows Python developers to write software that makes use of services like Amazon S3 and Amazon EC2.*
  * https://boto3.amazonaws.com/v1/documentation/api/latest/index.html  
  
*check what you have before installing/upgrading*  

In [ ]:
%%bash
python3 -m pip show boto3

In [ ]:
pip install -U boto3

### <p style="color:gold">3. aws configure</p> 
*Configure aws configure with credentials, and a user that has all of the Bedrock IAM policies required*  
https://docs.aws.amazon.com/bedrock/latest/userguide/security_iam_id-based-policy-examples.html
  
*You will need AWS CLI*  
https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-getting-started.html

In [ ]:
%%bash
aws sts get-caller-identity

<hr style="border:1px dotted;color:gold">